In [1]:
import pandas as pd

df = pd.read_csv('/data/ephemeral/home/ay_yolo/pseudo_label_7330_07.csv')
df.head()

,Unnamed: 0,PredictionString,image_id
0,0,1 0.8696467280387878 564.6475219726562 105.393...,test/0000.jpg
1,1,4 0.7968173027038574 343.303466796875 250.2890...,test/0001.jpg
2,2,1 0.8737943768501282 304.6231689453125 317.128...,test/0002.jpg
3,3,9 0.9233554005622864 144.21945190429688 262.50...,test/0003.jpg
4,4,1 0.8674394488334656 196.31341552734375 255.20...,test/0004.jpg


In [2]:
# NaN 값 확인
empty_rows_nan = df[df['PredictionString'].isnull()]
empty_bbox_image = list(empty_rows_nan.index)
# 결과 확인
print(empty_bbox_image)


[12, 33, 34, 38, 50, 63, 90, 123, 134, 140, 150, 173, 188, 208, 227, 237, 257, 262, 274, 327, 369, 438, 463, 468, 483, 487, 541, 566, 584, 638, 641, 650, 657, 673, 683, 713, 715, 717, 817, 837, 842, 870, 874, 907, 911, 926, 934, 963, 997, 1011, 1016, 1021, 1041, 1047, 1058, 1093, 1103, 1106, 1137, 1141, 1195, 1199, 1206, 1246, 1318, 1332, 1357, 1359, 1390, 1405, 1415, 1443, 1447, 1457, 1480, 1497, 1504, 1506, 1516, 1548, 1561, 1574, 1586, 1619, 1687, 1694, 1699, 1735, 1737, 1739, 1750, 1762, 1768, 1808, 1832, 1836, 1853, 1856, 1906, 1954, 1964, 1972, 1974, 1991, 2028, 2037, 2044, 2045, 2052, 2083, 2089, 2101, 2103, 2108, 2121, 2164, 2168, 2169, 2187, 2194, 2196, 2200, 2209, 2241, 2257, 2270, 2297, 2337, 2342, 2466, 2470, 2476, 2486, 2513, 2543, 2554, 2557, 2577, 2587, 2602, 2610, 2617, 2669, 2675, 2694, 2700, 2704, 2718, 2736, 2803, 2804, 2816, 2894, 2904, 2909, 2947, 2953, 2968, 2975, 2994, 3005, 3009, 3037, 3099, 3112, 3152, 3185, 3203, 3207, 3266, 3273, 3305, 3315, 3336, 3342, 3358,

In [3]:
df.dropna(subset=['PredictionString'], inplace=True)
df.reset_index(drop=True, inplace=True)

In [4]:
# PredictionString의 각 요소를 문자열로 변환하고, NaN은 빈 문자열로 대체한 후 띄어쓰기 기준으로 나누어 리스트로 변환
predict = df['PredictionString'].apply(lambda x: str(x).split())

# 결과 확인
print(predict)

0       [1, 0.8696467280387878, 564.6475219726562, 105...
1       [4, 0.7968173027038574, 343.303466796875, 250....
2       [1, 0.8737943768501282, 304.6231689453125, 317...
3       [9, 0.9233554005622864, 144.21945190429688, 26...
4       [1, 0.8674394488334656, 196.31341552734375, 25...
                              ...                        
4619    [1, 0.8271989226341248, 0.0, 415.8865051269531...
4620    [3, 0.9189334511756897, 415.57318115234375, 32...
4621    [7, 0.9177708029747009, 116.20901489257812, 44...
4622    [0, 0.8905738592147827, 0.483978271484375, 239...
4623    [4, 0.919512927532196, 373.9432373046875, 204....
Name: PredictionString, Length: 4624, dtype: object


In [5]:
# NaN 값 확인
empty_rows_nan = df[df['PredictionString'].isnull()]

# 결과 확인
print(empty_rows_nan)


Empty DataFrame
Columns: [Unnamed: 0, PredictionString, image_id]
Index: []


In [6]:
print(len(df)) #4853 -> 0.5일 때 Len

4624


In [7]:
import json

# JSON 파일 경로 설정
output_json_path = '/data/ephemeral/home/ay_yolo/yolo_dataset_ay/test.json'  # 결과를 저장할 파일

# test.json 읽기
with open(output_json_path, 'r') as f:
    test_data = json.load(f)

re_id = 4883
#bbox가 없는 test이미지 삭제
test_data['images'] = [image for image in test_data['images'] if image['id'] not in empty_bbox_image]
for i in range(len(test_data['images'])):
    test_data['images'][i]['id'] = re_id
    re_id += 1
    # print(test_data['images'][i]['id'])

In [8]:
new_test_data = []
start_id = 23144
for i in range(len(df)):
    predicts = predict[i]
    image_id = test_data['images'][i]['id'] #df['image_id'][i][5:9]
    id = int(image_id)
    for j in range(0,len(predicts),6):
        category_id = predicts[j]
        x1 = float(predicts[j+2])
        y1 = float(predicts[j+3])
        x2 = float(predicts[j+4])
        y2 = float(predicts[j+5])
        w = x2 - x1
        h = y2 - y1
        area = w*h
    
        new_item = {
        "image_id": id,
        "category_id" : int(category_id),
        "area" : area,
        "bbox" : [
            x1,
            y1,
            w,
            h
        ],
        "iscrowd" : 0,
        "id" : start_id
        }
        start_id += 1
        new_test_data.append(new_item)

In [9]:
new_test_data

[{'image_id': 4883,
  'category_id': 1,
  'area': 45521.053092492744,
  'bbox': [564.6475219726562,
   105.3935317993164,
   178.657470703125,
   254.79512786865234],
  'iscrowd': 0,
  'id': 23144},
 {'image_id': 4883,
  'category_id': 5,
  'area': 4232.960254177451,
  'bbox': [343.76824951171875,
   642.6324462890625,
   51.19573974609375,
   82.681884765625],
  'iscrowd': 0,
  'id': 23145},
 {'image_id': 4883,
  'category_id': 5,
  'area': 3734.4767384827137,
  'bbox': [244.34231567382812,
   683.6573486328125,
   57.370849609375,
   65.0936279296875],
  'iscrowd': 0,
  'id': 23146},
 {'image_id': 4883,
  'category_id': 7,
  'area': 99967.8762382567,
  'bbox': [216.11724853515625,
   51.718109130859375,
   238.2047119140625,
   419.672119140625],
  'iscrowd': 0,
  'id': 23147},
 {'image_id': 4883,
  'category_id': 7,
  'area': 179131.628215909,
  'bbox': [602.917236328125,
   517.8032836914062,
   354.1729736328125,
   505.7744140625],
  'iscrowd': 0,
  'id': 23148},
 {'image_id': 48

In [11]:
# import json

# JSON 파일 경로 설정
train_json_path = '/data/ephemeral/home/ay_yolo/yolo_dataset_ay/train_split_copy.json'


# train_split.json 읽기
with open(train_json_path, 'r') as f:
    train_data = json.load(f)

train_data['images'] += test_data['images']
train_data['annotations'] += new_test_data
    
# 수정된 내용을 JSON 파일로 저장
with open('train_plus_pseudo_label_0.7.json', 'w') as f:
    json.dump(train_data, f, indent=4)
print("pseudo_labeling을 위한 json파일 수정 끝~~~~")



pseudo_labeling을 위한 json파일 수정 끝~~~~


In [12]:
import os
import shutil

# pseudo_images 폴더 생성
pseudo_dir = '/data/ephemeral/home/ay_yolo/yolo_dataset_ay/pseudo07/images'
os.makedirs(pseudo_dir, exist_ok=True)

train_split_dir = '/data/ephemeral/home/ay_yolo/yolo_dataset_ay/train_split/images'

# train_split_dir의 이미지 파일들을 복사
for image_info in os.listdir(train_split_dir):
    # 파일의 ID 추출 (확장자 제외)
    src_img = os.path.join(train_split_dir, image_info)
    dst_path =  os.path.join(pseudo_dir,image_info)
    shutil.copy(src_img, dst_path)

print("train_split 이미지 복사가 완료되었습니다.")

train_split 이미지 복사가 완료되었습니다.


In [14]:
# test_dir 경로
test_dir = '/data/ephemeral/home/ay_yolo/yolo_dataset_ay/test'
t_dir = '/data/ephemeral/home/ay_yolo/yolo_dataset_ay'
pseudo_test_json_path = '/data/ephemeral/home/ay_yolo/train_plus_pseudo_label_0.7.json'


# train_split.json 읽기
with open(pseudo_test_json_path, 'r') as f:
    pseudo_test_data = json.load(f)

for i in range(len(pseudo_test_data["images"])):
    file_name = pseudo_test_data['images'][i]["file_name"]
    file_dir = file_name[0:4]
    
    if file_dir == "test":
        
        src_path = os.path.join(t_dir, file_name)
        new_id = pseudo_test_data['images'][i]["id"]
        new_id = str(new_id).zfill(4)
        # print(new_id)
        new_image_info = new_id+'.jpg'
        dst_path = os.path.join(pseudo_dir, new_image_info)
        shutil.copy(src_path, dst_path)

print("pseudo_labeling 이미지 복사가 완료되었습니다.")


4883
4884
4885
4886
4887
4888
4889
4890
4891
4892
4893
4894
4895
4896
4897
4898
4899
4900
4901
4902
4903
4904
4905
4906
4907
4908
4909
4910
4911
4912
4913
4914
4915
4916
4917
4918
4919
4920
4921
4922
4923
4924
4925
4926
4927
4928
4929
4930
4931
4932
4933
4934
4935
4936
4937
4938
4939
4940
4941
4942
4943
4944
4945
4946
4947
4948
4949
4950
4951
4952
4953
4954
4955
4956
4957
4958
4959
4960
4961
4962
4963
4964
4965
4966
4967
4968
4969
4970
4971
4972
4973
4974
4975
4976
4977
4978
4979
4980
4981
4982
4983
4984
4985
4986
4987
4988
4989
4990
4991
4992
4993
4994
4995
4996
4997
4998
4999
5000
5001
5002
5003
5004
5005
5006
5007
5008
5009
5010
5011
5012
5013
5014
5015
5016
5017
5018
5019
5020
5021
5022
5023
5024
5025
5026
5027
5028
5029
5030
5031
5032
5033
5034
5035
5036
5037
5038
5039
5040
5041
5042
5043
5044
5045
5046
5047
5048
5049
5050
5051
5052
5053
5054
5055
5056
5057
5058
5059
5060
5061
5062
5063
5064
5065
5066
5067
5068
5069
5070
5071
5072
5073
5074
5075
5076
5077
5078
5079
5080
5081
5082
